In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2e62qamk
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2e62qamk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=3258d565ad6627332ab67cacd393444b453a1d08995b58c1b48ae9a8debf9b05
  Stored in directory: /tmp/pip-ephem-wheel-cache-skeqd_88/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


**Load the Dataset and unzip it**

In [ ]:
!rm -r *
!wget -q https://www.dropbox.com/s/ic8ptf26601vbr5/Lab5.zip
!unzip Lab5.zip
!rm Lab5.zip
!ls

Archive:  Lab5.zip
   creating: Dataset/Test/
   creating: Dataset/Test/0/
  inflating: Dataset/Test/0/input.raw  
  inflating: Dataset/Test/0/MyOutput.raw  
  inflating: Dataset/Test/0/output.raw  
   creating: Dataset/Test/1/
  inflating: Dataset/Test/1/input.raw  
  inflating: Dataset/Test/1/MyOutput.raw  
  inflating: Dataset/Test/1/output.raw  
   creating: Dataset/Test/2/
  inflating: Dataset/Test/2/input.raw  
  inflating: Dataset/Test/2/MyOutput.raw  
  inflating: Dataset/Test/2/output.raw  
   creating: Dataset/Test/3/
  inflating: Dataset/Test/3/input.raw  
  inflating: Dataset/Test/3/MyOutput.raw  
  inflating: Dataset/Test/3/output.raw  
   creating: Dataset/Test/4/
 extracting: Dataset/Test/4/input.raw  
  inflating: Dataset/Test/4/MyOutput.raw  
  inflating: Dataset/Test/4/output.raw  
   creating: Dataset/Test/5/
  inflating: Dataset/Test/5/input.raw  
  inflating: Dataset/Test/5/MyOutput.raw  
  inflating: Dataset/Test/5/output.raw  
   creating: Dataset/Test/6/
  infla

# **CUDA BASE Code Using Global Memory Version**

Code to be modified where there are TODO indications

In [ ]:
%%writefile lab5-BASE.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#define NUM_BINS 4096


__global__ void histogram_kernel(unsigned int *input, unsigned int *bins, unsigned int num_elements, unsigned int num_bins) {

	unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;

	// TODO write histogram code using global memory
	int stride = blockDim.x * gridDim.x;
	while (tid < num_elements) {
		atomicAdd(&(bins[input[tid]]), 1);
		tid += stride;
	}
}
__global__ void convert_kernel(unsigned int *bins, unsigned int num_bins) {

	unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;

	if (tid < num_bins) {
		bins[tid] = min(bins[tid], 127);
	}
}

void histogram(unsigned int *input, unsigned int *bins, unsigned int num_elements, unsigned int num_bins) {
  cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	// zero out bins
	cudaMemset(bins, 0, num_bins * sizeof(unsigned int));
	// Launch histogram kernel on the bins
	{
		cudaEventCreate(&start);
	  cudaEventCreate(&stop);
	  cudaEventRecord(start, 0);
		dim3 blockDim(512), gridDim(30);
		histogram_kernel << <gridDim, blockDim, num_bins * sizeof(unsigned int) >> >(input, bins, num_elements, num_bins);
		cudaEventRecord(stop, 0);
	  cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&gpu_time, start, stop);
	  printf("-------->Histogram Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	}

	// Make sure bin values are not too large
  {
	  dim3 blockDim(512);
	  dim3 gridDim((num_bins + blockDim.x - 1) / blockDim.x);
		cudaEventRecord(start, 0);
	  convert_kernel << <gridDim, blockDim >> >(bins, num_bins);
	  cudaEventRecord(stop, 0);
	  cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&gpu_time, start, stop);
	  printf("-------->Convert Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);
  }
	cudaEventDestroy(start);
	cudaEventDestroy(stop);
}


unsigned int* ImportRawInteger(char* filename, int* N)
{
	FILE* handle;
	unsigned int* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return 0;
	}
	fscanf(handle, "%d", N);
	val = (unsigned int*)malloc(*N * sizeof(unsigned int));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%d", val + i);
	}
	fclose(handle);
	return val;

}
unsigned int ExportRawInteger(char* filename, unsigned int* val, int N)
{
	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}
	handle = fopen(filename, "w");

	if (handle == NULL) {
		printf("Error opening file: %s\n", filename);
		return 0;
	}

	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%d\n", val[i]);
	}
	fclose(handle);
	return 1;
}

clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

int main(int argc, char **argv) {
	int inputLength;
	unsigned int *hostInput;
	unsigned int *hostBins;
	unsigned int *hostExpected;
	unsigned int *deviceInput;
	unsigned int *deviceBins;
	int i,prt = 0;
	int M;
	float meanDiff = 0;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[3];

for (ti=0;ti<7;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv4,"ON");

	printf("Running GPU Histogram...\n");
	if (strcmp(argv4, "ON") == 0)prt = 1;

	StartingTime = Time_start();
	hostExpected = (unsigned int*)ImportRawInteger(argv1, &M);
	hostInput = (unsigned int*)ImportRawInteger(argv2, &inputLength);

	Elapsed_time(StartingTime, "Importing data and creating memory on host", prt);
	printf("The input length is %d\n", inputLength);


	hostBins = (unsigned int *)malloc(NUM_BINS * sizeof(unsigned int));

	/*TODO: Allocate GPU Memory*/
	StartingTime=Time_start();
	cudaMalloc(&deviceInput, inputLength * sizeof(unsigned int));
	cudaMalloc(&deviceBins, NUM_BINS * sizeof(unsigned int));

	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
	cudaMemcpy(deviceInput, hostInput, inputLength * sizeof(unsigned int), cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here
	dim3 gridDim(ceil(inputLength/32)+1, 1, 1);
	dim3 blockDim(32, 1, 1);
	StartingTime = Time_start();

	histogram(deviceInput, deviceBins, inputLength, NUM_BINS);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
	cudaMemcpy(hostBins, deviceBins, NUM_BINS*sizeof(unsigned int), cudaMemcpyDeviceToHost);
	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
	cudaFree(deviceBins);
	cudaFree(deviceInput);
	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);

	// Verify correctness
	// -----------------------------------------------------
	for (i = 0; i < M; i++) {
		meanDiff = meanDiff + fabs((float)hostBins[i] - (float)hostExpected[i]);
	}
	meanDiff = meanDiff / (float)M;
	if (meanDiff > 0.01) {
		printf("%f Failed\n", meanDiff);

	}
	else {
		printf("Passed\n");
	}
	ExportRawInteger(argv3, hostBins, M);

	free(hostBins);
	free(hostInput);
}

#if LAB_DEBUG
	system("pause");
#endif

	return 0;
}


Writing lab5-BASE.cu


Compile the CPU program using gcc compiler

In [ ]:
!nvcc  lab5-BASE.cu -o lab5-BASE.exe
!ls

lab5-BASE.cu(179): warning: variable "blockDim" was declared but never referenced

Dataset  lab5-BASE.cu  lab5-BASE.exe


**Test CPU code execution**

In [ ]:
!./lab5-BASE.exe



 +++++++++++++Test 0
Running GPU Histogram...
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input.raw
Importing data and creating memory on host Elapsed Time 0.624000 in mili-seconds
The input length is 16
Allocate GPU Memory Elapsed Time 188.313000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.025000 in mili-seconds
Launching kernel
-------->Histogram Kernel Elapsed Time*********** 0.013760 in mili-seconds
-------->Convert Kernel Elapsed Time*********** 0.010240 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.038000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.138000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Running GPU Histogram...
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input.raw
Importing data and creating memory on host Elapsed Time 0.403000 in mili-seconds
The input length is 1024
Allocate GPU Memory Elapsed Time 0.146000 in mili-seconds
Copying 

Save trace into Lab5Base.txt

# **GPU Code with Private Copy in Shared Memory**

In [ ]:
%%writefile lab5-PRIVATE.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#define NUM_BINS 4096


__global__ void histogram_kernel_shared(unsigned int *input, unsigned int *bins, unsigned int num_elements, unsigned int num_bins) {

	unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;

	// TODO write histogram code using private shared memory
	extern __shared__ unsigned int histogram_private[];

	unsigned int i = threadIdx.x;
	while (i < num_bins) {
		histogram_private[i] = 0;
		i += blockDim.x;
	}

	__syncthreads();

	int stride = blockDim.x * gridDim.x;
	while (tid < num_elements) {
		atomicAdd(&(histogram_private[input[tid]]), 1);
		tid += stride;
	}

	__syncthreads();

	i = threadIdx.x;
	while (i < num_bins) {
		atomicAdd(&(bins[i]), histogram_private[i]);
		i += blockDim.x;
	}

}

__global__ void convert_kernel(unsigned int *bins, unsigned int num_bins) {

	unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;

	if (tid < num_bins) {
		bins[tid] = min(bins[tid], 127);
	}
}

void histogram(unsigned int *input, unsigned int *bins, unsigned int num_elements, unsigned int num_bins) {
  cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	// zero out bins
	cudaMemset(bins, 0, num_bins * sizeof(unsigned int));
	// Launch histogram kernel on the bins
	{
		cudaEventCreate(&start);
	  cudaEventCreate(&stop);
	  cudaEventRecord(start, 0);
		dim3 blockDim(512), gridDim(30);
		histogram_kernel_shared  << <gridDim, blockDim, num_bins * sizeof(unsigned int) >> >(input, bins, num_elements, num_bins);
		cudaEventRecord(stop, 0);
	  cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&gpu_time, start, stop);
	  printf("-------->Histogram Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	}

	// Make sure bin values are not too large
  {
	  dim3 blockDim(512);
	  dim3 gridDim((num_bins + blockDim.x - 1) / blockDim.x);
		cudaEventRecord(start, 0);
	  convert_kernel << <gridDim, blockDim >> >(bins, num_bins);
	  cudaEventRecord(stop, 0);
	  cudaEventSynchronize(stop);
	  cudaEventElapsedTime(&gpu_time, start, stop);
	  printf("-------->Convert Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);
  }
	cudaEventDestroy(start);
	cudaEventDestroy(stop);
}


unsigned int* ImportRawInteger(char* filename, int* N)
{
	FILE* handle;
	unsigned int* val;
	int i;
	printf("Reading File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}

	handle = fopen(filename, "r");
	if (handle == NULL) {
		printf("Failed to open %s\n", filename);
		return 0;
	}
	fscanf(handle, "%d", N);
	val = (unsigned int*)malloc(*N * sizeof(unsigned int));
	for (i = 0; i < *N; i++) {
		fscanf(handle, "%d", val + i);
	}
	fclose(handle);
	return val;

}
unsigned int ExportRawInteger(char* filename, unsigned int* val, int N)
{
	FILE* handle;
	int i;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}
	handle = fopen(filename, "w");

	if (handle == NULL) {
		printf("Error opening file: %s\n", filename);
		return 0;
	}

	fprintf(handle, "%d\n", N);
	for (i = 0; i < N; i++) {
		fprintf(handle, "%d\n", val[i]);
	}
	fclose(handle);
	return 1;
}

clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

int main(int argc, char **argv) {
	int inputLength;
	unsigned int *hostInput;
	unsigned int *hostBins;
	unsigned int *hostExpected;
	unsigned int *deviceInput;
	unsigned int *deviceBins;
	int i,prt = 0;
	int M;
	float meanDiff = 0;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[3];

for (ti=0;ti<7;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv4,"ON");

	printf("Running GPU Histogram...\n");
	if (strcmp(argv4, "ON") == 0)prt = 1;

	StartingTime = Time_start();
	hostExpected = (unsigned int*)ImportRawInteger(argv1, &M);
	hostInput = (unsigned int*)ImportRawInteger(argv2, &inputLength);

	Elapsed_time(StartingTime, "Importing data and creating memory on host", prt);
	printf("The input length is %d\n", inputLength);


	hostBins = (unsigned int *)malloc(NUM_BINS * sizeof(unsigned int));

	/*TODO: Allocate GPU Memory*/
	StartingTime=Time_start();
	cudaMalloc(&deviceInput, inputLength * sizeof(unsigned int));
	cudaMalloc(&deviceBins, NUM_BINS * sizeof(unsigned int));


	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
	cudaMemcpy(deviceInput, hostInput, inputLength * sizeof(unsigned int), cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here
	dim3 gridDim(ceil(inputLength/32)+1, 1, 1);
	dim3 blockDim(32, 1, 1);
	StartingTime=Time_start();

	histogram(deviceInput, deviceBins, inputLength, NUM_BINS);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
	cudaMemcpy(hostBins, deviceBins, NUM_BINS * sizeof(unsigned int), cudaMemcpyDeviceToHost);
	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
	cudaFree(deviceBins);
	cudaFree(deviceInput);
	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);

	// Verify correctness
	// -----------------------------------------------------
	for (i = 0; i < M; i++) {
		meanDiff = meanDiff + fabs((float)hostBins[i] - (float)hostExpected[i]);
	}
	meanDiff = meanDiff / (float)M;
	if (meanDiff > 0.01) {
		printf("%f Failed\n", meanDiff);

	}
	else {
		printf("Passed\n");
	}
	ExportRawInteger(argv3, hostBins, M);

	free(hostBins);
	free(hostInput);
}

#if LAB_DEBUG
	system("pause");
#endif

	return 0;
}


Writing lab5-PRIVATE.cu


**Compile the CUDA code using shared memory**

In [ ]:
!nvcc  lab5-PRIVATE.cu -o lab5-PRIVATE.exe
!ls

lab5-PRIVATE.cu(200): warning: variable "blockDim" was declared but never referenced

Dataset  lab5-BASE.cu  lab5-BASE.exe  lab5-PRIVATE.cu  lab5-PRIVATE.exe


**Execute CUDA Code using shared memory**

In [ ]:
!./lab5-PRIVATE.exe



 +++++++++++++Test 0
Running GPU Histogram...
Reading File Dataset/Test/0/output.raw
Reading File Dataset/Test/0/input.raw
Importing data and creating memory on host Elapsed Time 0.449000 in mili-seconds
The input length is 16
Allocate GPU Memory Elapsed Time 156.094000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.032000 in mili-seconds
Launching kernel
-------->Histogram Kernel Elapsed Time*********** 0.017408 in mili-seconds
-------->Convert Kernel Elapsed Time*********** 0.009568 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.036000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.134000 in mili-seconds
Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Running GPU Histogram...
Reading File Dataset/Test/1/output.raw
Reading File Dataset/Test/1/input.raw
Importing data and creating memory on host Elapsed Time 0.702000 in mili-seconds
The input length is 1024
Allocate GPU Memory Elapsed Time 0.173000 in mili-seconds
Copying 